In [2]:
import gensim
import corenlp
import spacy


In [3]:
parser = spacy.load('en')

In [4]:
text = open('article.txt').read()
chunks  = text.split('\n\n')

In [5]:
def serious_analysis(text, parser):
    doc = parser(text)
    for token in doc:
        print (token, token.pos_, token.head, token.dep_, '(',token.ent_type,')', token.lemma_, token.sentiment)



In [66]:
from IPython.core.display import HTML

def highlight_chunk(text, parser):
    doc = parser(text)

    from IPython.core.display import HTML
    s = ""
    #word_wrap = '<style>p {width: 420px; white-space: break-word; overflow: hidden; text-overflow: ellipsis; } </style> <p id=\"formatted_text\">'
    #s+= word_wrap

    for token in doc:
        
        token_string = token.text_with_ws
        
        if token.dep_ in ['ROOT']:
            t='<font color="#ff0000">' +token_string + '</font>'
        elif token.dep_ in ['nsubj','nsubjpass', 'csubj']:
            t='<font color="#0000ff">' +token_string + '</font>'
        elif token.dep_ in ['compound', 'advmod'] and token.head.dep_ in ['nsubj']:
            t='<font color="#2255dd">' +token_string + '</font>'
        elif token.dep_ in ['pobj', 'dobj']:
            t='<font color="#33aa11">' +token_string + '</font>'
        elif token.pos_ in ['VERB', 'ADVMOD'] or token.dep_ in ['neg', 'prt']:
            t='<font color="#ff0000">' +token_string + '</font>'
            """
            elif token.dep_ in ['dobj'] or token.pos_ in []:
                s+='<font color="#0000ff">' +token.text_with_ws + '</font>'
            elif token.dep_ in [] or token.pos_ in ['ADV']:
                s+='<font color="#ff0077">' +token.text_with_ws + '</font>'
            elif token.dep_ in [] or token.pos_ in ['ADJ']:
                s+='<font color="#00ffff">' +token.text_with_ws + '</font>'
            """
        else:
            t = '<font color="#444444">' +token_string + '</font>'

        s +=t
    s=s.replace('\\n','<br>')
    #s+='</p>'
    #HTML(s+"<br><br><br>")
    return s
    
def format_chunk(text):
    doc = parser(text)

    from IPython.core.display import HTML
    s = ""
    word_wrap = '<style>p {width: 420px; white-space: break-word; overflow: hidden; text-overflow: ellipsis; } </style> <p>'
    s+= word_wrap
    
    s+=text
    
    s=s.replace('\\n','<br>')
    s+='</p>'
    #HTML(s+"<br><br><br>")
    return s

In [54]:
idx = 0

In [55]:
idx = idx + 1

In [56]:
HTML("<style>p {width: 420px; white-space: break-word; overflow: hidden; text-overflow: ellipsis; } </style> <p>'"+chunks[idx])

In [57]:
HTML(highlight_chunk(chunks[idx], parser))


NameError: name 'word_wrap' is not defined

In [58]:
serious_analysis(chunks[idx], parser)

You PRON putting nsubj ( 0 ) -PRON- 0.0
’ve VERB putting aux ( 0 ) have 0.0
been VERB putting aux ( 0 ) be 0.0
putting VERB putting ROOT ( 0 ) put 0.0
off PART putting prt ( 0 ) off 0.0
reading NOUN putting advcl ( 0 ) reading 0.0
that DET book det ( 0 ) that 0.0
book NOUN reading dobj ( 0 ) book 0.0
for ADP putting prep ( 0 ) for 0.0
weeks NOUN for pobj ( 388 ) week 0.0
, PUNCT putting punct ( 0 ) , 0.0
and CCONJ putting cc ( 0 ) and 0.0
you PRON supposed nsubjpass ( 0 ) -PRON- 0.0
’re VERB supposed auxpass ( 0 ) be 0.0
supposed VERB putting conj ( 0 ) suppose 0.0
to PART read aux ( 0 ) to 0.0
have VERB read aux ( 0 ) have 0.0
read VERB supposed xcomp ( 0 ) read 0.0
it PRON read dobj ( 0 ) -PRON- 0.0
all DET read npadvmod ( 0 ) all 0.0
by ADP read prep ( 0 ) by 0.0
tomorrow NOUN by pobj ( 388 ) tomorrow 0.0
. PUNCT supposed punct ( 0 ) . 0.0
Whether ADP cramming mark ( 0 ) whether 0.0
you PRON cramming nsubj ( 0 ) -PRON- 0.0
’re VERB cramming aux ( 0 ) be 0.0
cramming VERB lose csubj 

In [59]:
highlight_chunk(chunks[2], parser)

NameError: name 'word_wrap' is not defined

In [60]:

from spacy import displacy
def display_parse(text, parser):
    doc = parser(text)
    return displacy.render(doc, style='dep')

In [61]:
HTML(displacy.render(parser(chunks[5]), style='dep'))

In [62]:
chunk = "A remarkable aspect of your mental life is that you are rarely stumped … The normal state of your mind is that you have intuitive feelings and opinions about almost everything that comes your way. You like or dislike people long before you know much about them; you trust or distrust strangers without knowing why; you feel that an enterprise is bound to succeed without analyzing it."

In [63]:
HTML(highlight_chunk(chunk, parser))


NameError: name 'word_wrap' is not defined

In [64]:
HTML(format_chunk(chunk))

In [ ]:
from flask import Flask, render_template, request, jsonify
from jinja2 import Template
from jinja2 import evalcontextfilter, Markup, escape
import re

app = Flask(__name__)

@app.template_filter()
@evalcontextfilter
def nl2br(eval_ctx, value):
   _paragraph_re = re.compile(r'(?:\r\n|\r(?!\n)|\n){2,}')
   result = u'\n\n'.join(u'<p>%s</p>' % p.replace(u'\r\n', u'<br/>') for p in _paragraph_re.split(value))
   if eval_ctx.autoescape:
       result = Markup(result)
   return result

@app.route("/")
def one():
    return render_template('index.html')

@app.route('/step2', methods=['POST'])
def two():
    formatted_text = highlight_chunk(request.form['text-content'], parser) 
    return render_template('step2.html', formatted_text = formatted_text, 
                                         email = request.form['email'])

@app.route('/step3', methods=['POST'])
def three():
    return render_template('step3.html', this_WPM = 111,
                          articles = [
                              {'date':'01/01/2018', 'n_words':500, 'speed':111},
                              {'date':'02/01/2018', 'n_words':600, 'speed':112},
                              {'date':'03/01/2018', 'n_words':700, 'speed':113},
                              {'date':'04/01/2018', 'n_words':800, 'speed':114},
                            
                           ]
                          )


app.host='0.0.0.0'
app.use_reloader = False
app.debug = False
app.run(host='0.0.0.0')



 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Mar/2018 20:34:26] "POST /step2 HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:34:26] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:34:26] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:34:28] "POST /step3 HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:35:21] "POST /step3 HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:37:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:37:28] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:37:28] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:37:47] "POST /step2 HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:37:47] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:37:47] "GET /assets/js/scripts.js HTTP/1.1" 404 -
127.0.0.1 - - [20/Mar/2018 20:38:16] "POST /step3 HTTP/1.1" 200 -
127.0.0.1 - - [20/Mar/2018 20:39:44] "POST /step2 HTTP/1.1" 200 -
127.0.